In [ ]:
# pip install nltk

In [ ]:
# pip install sklearn

In [ ]:
# pip install -U scikit-learn

In [2]:
import csv
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option('display.max_colwidth', 300)

In [6]:
df = pd.read_csv('all_cuisine_df.csv')

In [ ]:
df_big = pd.read_csv('replenish/Food Ingredients and Recipe Dataset with Image Name Mapping.csv')

In [ ]:
def text_cleaning(text):
    text = str(text)
    for punc in string.punctuation:
        text = text.replace(punc, ' ')
    text = text.lower()
    only_alpha = ' '.join([t for t in text.split() if t.isalpha()])
    only_alpha = only_alpha.replace('lb', '').replace('Tbsp', '').replace('tsp', '').replace('oz', '').replace('cup', '').replace('to', '')
    only_alpha = only_alpha.strip()
    only_alpha = only_alpha.lower()
    only_alpha = ' '.join(only_alpha.split())
    return only_alpha

In [ ]:
df.head(1)

In [86]:
customised_sw =['score', 'teaspoon', 'tablespoons', 'chopped', 'fresh',
                'tablespoon', 'large', 'ground', 'teaspoons', 'finely', 
                'ounces', 'sliced', 'kosher', 'plus', 'cut', 'inch', 
                'divided', 'juice', 'unsalted', 'black', 'grated', 
                'freshly', 'pound', 'red', 'peeled', 'thinly', 'ounce', 
                'cream', 'leaves', 'white', 'whole', 'pounds', 'extra', 
                'vinegar', 'medium', 'water', 'small', 'virgin','purpose', 
                'pieces', 'vegetable', 'coarsely', 'dried', 'seeds', 
                'powder', 'halved', 'minced', 'sauce', 'baking', 'packed', 
                'stick', 'serving', 'green', 'thick', 'slices', 'extract', 
                'optional', 'dry', 'temperature', 'room', 'brown', 'trimmed', 
                'sea', 'heavy', 'crushed', 'lengthwise', 'zest', 'toasted', 
                'removed', 'cubes', 'preferably', 'diced', 'drained', 'fren', 
                'chilled', 'broth', 'light', 'coarse', 'wedges', 'syrup', 'leaf', 
                'pinch', 'garnish', 'sticks', 'taste', 'hot', 'unsweetened', 'flakes',
                'sweet', 'crosswise', 'stems', 'half','melted', 'dark', 
                'granulated', 'low', 'quartered', 'pitted', 'flat', 'pan', 'torn', 'pure',
                'seeded', 'rinsed', 'baby', 'peel', 'italian', 'bunch', 'fine', 'yellow',
                'fat', 'paste', 'plain', 'head', 'ripe', 'bay', 'sodium', 'boneless', 
                'sour', 'lightly', 'cider', 'softened', 'strips', 'thawed', 'whipping',
                'crumbled', 'thin', 'bell', 'smoked','yolks', 'beaten', 'cooked', 
                'dijon', 'ml', 'package', 'cored', 'piece', 'shredded', 'spray',
                'golden', 'skin', 'roasted', 'raw', 'bittersweet', 'tbsp', 'powdered', 
                'total', 'fillets', 'additional', 'grams', 'buttermilk', 'cold', 
                'cornstarch', 'ribs', 'long', 'maple', 'style', 'separated', 'skinless',
                'flaky', 'stock', 'bone', 'nonstick', 'greek', 'balsamic', 'stalks', 
                'mixed', 'grain', 'canned', 'parts', 'bean', 'greens', 'sage', 'drizzling',
                'see', 'wheat', 'tender', 'whites', 'frying', 'scrubbed', 'chips', 'unpeeled',
                'bunches', 'split', 'canola', 'squash', 'one', 'firm', 'cooking', 'gold', 
                'reduced', 'fish', 'de', 'well', 'equipment', 'dusting', 'stemmed', 
                'thermometer', 'deep', 'crème', 'size', 'asian', 'rounds', 'pastry', 
                'quality', 'inches', 'spice', 'shelled', 'accompaniment', 'semisweet', 
                'allspoon', 'evaporated', 'breast', 'boiling', 'cracked', 'needed', 
                'diagonal', 'supermarkets', 'soft', 'foods', 'stores', 'jar', 
                'food', 'whipped', 'old', 'english', 'brandy', 'distilled', 
                'homemade', 'quart', 'loosely', 'loaf', 'markets', 'worcestershire',  
                'sprinkling', 'regular', 'hulled', 'center', 'sharp', 'unbleached',
                'assorted', 'sifted', 'dice', 'seedless', 'chops', 'free', 'scant', 
                'herbs', 'short', 'spanish', 'seasoning', 'fillet', 'prepared', 
                'use', 'ingredient', 'florets', 'pods', 'simple', 'blanched', 
                'info', 'shoulder', 'good', 'active', 'wild', 'packages', 'grill', 
                'tart', 'shaved', 'parchment', 'french', 'cubed', 'hard', 'breasts', 
                'tough', 'fresno', 'rings', 'bitters', 'sheets', 'minutes', 
                'pressed', 'star', 'desired', 'roast', 'pink', 'mix', 'crust', 
                'stalk', 'chinese', 'maldon', 'bite', 'dish', 'turkish', 'tea', 
                'club', 'soaked', 'bottled', 'fronds', 'slightly', 'high', 'candy', 
                'skinned', 'handful', 'flavored', 'california', 'seasoned', 'blade', 
                'meat', 'broken', 'ends', 'swiss', 'pints', 'two', 'dates',
                'espresso', 'greasing', 'mill', 'blue', 'new', 'romaine', 'cookies', 
                'saffron', 'cured', 'tops', 'left', 'specialty', 'unflavored', 
                'cleaned', 'brine', 'smith', 'granny', 'boiled', 'scallion', 'puff', 
                'cooks', 'romano', 'recipe', 'bags', 'generous', 'quarts', 'fashioned', 
                'sheet', 'surface', 'marjoram', 'persian', 'sparkling', 'bus',
                'adobo', 'fruit', 'end', 'chard', 'metal', 'box', 'diameter', 'slivered', 
                'heirloom', 'beer', 'purchased', 'mexican',
                'mini', 'bones', 'husked', 'picked', 'kitchen', 'fl',
                'preserves', 'skewers', 'shell', 'hearts',
                'dashes', 'container', 'serve', 'hothouse', 'chuck', 'nam', 
                'pizza', 'ancho', 'pickled', 'sandwich', 
                'uncooked', 'slice', 'jack', 'including', 'called', 'also', 
                'rolls', 'pestle', 'bottom', 'mortar', 'casings', 
                'cutter', 'tartar', 'part', 'rind', 'sunflower', 'wooden',
                'pans', 'muffin', 'árbol', 'threads',
                'crystal', 'condensed', 'cannellini', 'strip', 
                'matchsticks', 'intact', 'aleppo', 'shortening', 'buns',
                'crusty', 'top', 'flaked', 'navel',
                'legs', 'pearl', 'quick', 'julienned', 
                'angostura', 'com', 'read', 'snap', 'blood', 'nonfat',
                'made', 'mashed', 'caraway', 'meal', 'fry', 'sourdough', 'graham',
                'crimini', 'lemongrass', 'coloring', 'outer',
                'slicer', 'diamond', 'dutch', 'like', 'creamy', 'attached',
                'spicy', 'meyer', 'rose', 'candied', 'lean', 'tenderloin',
                'firmly', 'fontina', 'sun', 'adjustable', 'shucked', 'loin', 
                'basmati', 'envelope', 'monterey', 'peeler', 'liquid', 'full',
                'warmed', 'eye', 'tip', 'butt', 'washed', 'korean', 'pepitas', 
                'maker', 'aged', 'boston', 'processor', 'side', 'springform', 
                'fitted', 'bowl', 'electric', 'dash', 'spring', 'roasting', 
                'four', 'mandoline', 'tuscan', 'leg', 'crystallized', 'wings', 
                'twist', 'cheesecloth', 'neutral', 'grade', 'diagonally', 
                'accompaniments', 'core', 'stem', 'brand', 'drops', 'using', 
                'rising', 'penne', 'cremini', 'plastic', 'allspice', 'lady',
                'sturdy', 'eow', 'round', 'qt', 'shallow', 'ceramic', 'glass',
                'reggiano', 'storebought', 'reposado', 'agave', 'grand', 'pat', 'depending',
                'wheels', 'salted', 'sweetness', 'amaro', 'marnier', 'averna', 'dehydrated',
                'wheel', 'amontillado','steamed', 'skirt', 'mixer', 'concentrate', 'rolled', 'containers',
                'poblano', 'glass', 'shaken', 'sometimes', 'grilled', 'found',
                'jasmine', 'jumbo', 'scotch', 'escarole', 'horizontally', 'peach',
                'self', 'gala', 'stone', 'rounded', 'flowers', 'drumsticks',
                'least', 'vegetables', 'removable', 'shells', 'frisée', 'grinder',
                'skillet', 'string', 'strained', 'percent', 'sel',
                'seed', 'pot', 'morton', 'fingerling',
                'kidney', 'wedge', 'knife', 'flank', 'grand', 'delicious', 'tomatillos',
                'process', 'marnier', 'herb', 'spears', 'poppy', 'purple',
                'cane', 'button', 'matzo', 'skim', 'square', 'heaping',
                'purée', 'fleur', 'racks', 'lengths', 'qt', 'necessary', 'choice',
                'semolina', 'barbecue', 'sirloin', 'neck', 'topping',
                'cocktail', 'napa', 'fresco', 'ruby', 'rack', 'section', 'clean',
                'shaped', 'chia', 'choy', 'hoisin', 'salad', 'crisp', 'overnight', 
                'cm', 'jars', 'pla', 'harissa', 'tips', 'hungarian', 'port', 'shavings', 
                'little', 'masala', 'vidalia', 'molds', 'verts', 'grater', 'endive', 
                'stand', 'excess', 'habanero', 'lump', 'yams', 'another', 'prunes', 
                'favorite', 'melon', 'per', 'phyllo', 'palm', 'wheel', 'tamari', 
                'notes', 'stirred', 'cointreau', 'belgian', 'refrigerated', 'gluten', 
                'kimchi', 'puree', 'caps', 'littleneck', 'substitute', 'sumac', 'ghee', 
                'chestnuts', 'peppermint', 'glaze', 'bok', 'table', 'bibb', 'bottles', 
                'caramel', 'grilling', 'farro', 'beet', 'day', 'curly', 'lady', 'haricots', 
                'lowfat', 'bosc', 'five', 'spatula', 'baked', 'clam', 'chickens', 
                'crustless', 'pickles', 'iron', 'brewed', 'jelly', 'guajillo', 'strong', 
                'smooth', 'weights', 'attachment', 'garam', 'daikon', 'dulce', 'solid', 
                'tamarind', 'nuoc', 'back', 'bits', 'wax', 'starch', 'range', 'fluted', 
                'squares', 'couscous', 'rolling', 'sanding', 'labeled', 'combination', 
                'slab', 'tails', 'foil', 'skins', 'links', 'tenderloins', 'iceberg', 'lindt', 'fava',
                'layers', 'tied', 'sambal', 'tube', 'leaving', 'latin',
                'juices', 'online', 'salata', 'less',
                'heart', 'thigh', 'grits', 'cutlets', 'crusts', 'live', 'sake',
                'double', 'chervil', 'pea', 'pop', 'shallow',
                'matchstick', 'lager', 'measuring', 'hour', 'best', 'middle',
                'sold', 'tapioca', 'giblets', 'savoy',
                'wheels', 'balls', 'hamburger', 'oelek', 'depending', 'coating', 'ball',
                'young', 'breakfast', 'color', 'orzo', 'block',
                'leftover', 'fuji', 'anjou', 'seltzer', 'thickly',
                'demerara', 'plate', 'grind', 'better', 'cracker',
                'sized', 'cast', 'american', 'envelopes', 'rotisserie',
                'cube', 'indian', 'duty', 'slender', 'preserved', 'exceed',
                'castelvetrano', 'wrappers', 'special', 'quarters', 'blanc',
                'vegan', 'frenched', 'rabe', 'offset', 'cotija', 'edible', 'eastern',
                'heat', 'honeydew', 'acting', 'chanterelle', 'juniper', 'many', 'turbinado', 
                'husks', 'still', 'lavender', 'snapper', 'mahi', 'beefsteak', 'malt', 'wafer', 
                'kabocha', 'york', 'spelt', 'dressing', 'milliliter', 'fried', 'blanco',
                'caster', 'blackstrap', 'basic', 'butterflied', 'bartlett',
                'lima', 'kg', 'spiced', 'summer', 'crema', 'pinches', 'triple',
                'larger', 'evaporated', 'striped', 'marinated', 'challah', 'cooker',
                'fruity', 'without', 'debearded', 'hibiscus', 'batch',
                'semi', 'snow', 'grainy', 'aluminum', 'wrapped', 'sichuan', 'relish',
                'perugina', 'three', 'currant', 'julienne', 'layer', 'key',
                'splash', 'casing', 'twists', 'bonnet', 'thirds', 'flower',
                'rimmed', 'matcha', 'okra', 'provolone', 'moons',
                'premium', 'st', 'rigatoni', 'calvados', 'sec', 'lukewarm',
                'blossoms', 'pod', 'mezcal', 'oven', 'jicama', 'bella', 'fully',
                'leche', 'marcona', 'portobello', 'maitake', 'kaffir', 'mixture',
                'gochujang', 'usually', 'flaxseed', 'disposable', 'jarred', 'tin',
                'holes', 'recipelink', 'andouille', 'bias', 'ribbons', 'ricer', 
                'clear', 'colors', 'mission', 'sushi', 'gorgonzola', 'pinto', 'delicata',
                'bamboo', 'winter', 'kombu', 'fleshed', 'imported',
                'king', 'candies', 'tentacles', 'backbone', 'kumquats',
                'san', 'scraped', 'meaty', 'pullman', 'eight',
                'marinara', 'amaretto', 'whisked', 'wafers', 'spoon',
                'bulgur', 'according', 'stout', 'mexico', 'pounded', 'asiago',
                'segments', 'wood', 'unfiltered', 'shaoxing', 'drumstick', 'verde',
                'lid', 'sorbet', 'wiped', 'roll', 'pith', 'paddle', 'nori',
                'flameproof', 'kirby', 'ear', 'flesh', 'celeriac', 'several',
                'chickpea', 'bowls', 'dipping', 'bucatini', 'tail', 'passion', 'tiny',
                'six', 'gochugaru', 'soba', 'gem', 'boned', 'applewood', 'mung',
                'orecchiette', 'variety', 'reserving', 'rustic', 'bodies', 'safflower',
                'colored', 'nut', 'strings', 'shanks', 'bar', 'curd', 'log', 'work',
                'tomatillo', 'amaretti', 'amber', 'microplane', 'cassis', 'defrosted',
                'eyed', 'look', 'den', 'shank', 'la', 'blossom', 'sauvignon', 'marsala',
                'sprinkles', 'robust', 'loose', 'schmaltz', 'shoots', 'blender', 'make',
                'thaw', 'silken', 'fire', 'slow', 'icing', 'whisk',
                'ale', 'bitter', 'rainbow', 'buffalo', 'gram',
                'elderflower', 'juiced', 'sieve', 'press', 'masa', 'gouda', 'marzano',
                'sides', 'pin', 'bundt', 'big', 'need', 'honeycrisp',
                'cutters', 'rendered', 'nigella', 'walla', 'citrus', 'pernod', 'roots',
                'flavor', 'macadamia', 'aperol', 'breads', 'manila', 'hemp', 'known',
                'macaroni', 'persimmons', 'decorating', 'maui', 'irish',
                'hours', 'added', 'aniseed', 'standard', 'directions',
                'spread', 'brush', 'di', 'muffins', 'packaged', 'verbena',
                'malted', 'single', 'bonito', 'gum', 'charcoal', 'sorghum', 'herbes',
                'reserve', 'arctic', 'pickling', 'madras', 'vine',
                'first', 'crispy', 'based', 'fluid', 'gray', 'pits',
                'treviso', 'dissolved', 'frosting', 'rub',
                'marked', 'niçoise', 'bottomed', 'add', 'chartreuse', 'pectin', 'pulp',
                'bird', 'slaw', 'dredging', 'cornbread', 'wonton', 'stuffed',
                'guanciale', 'milliliters', 'mature', 'sift', 'endives',
                'bouillon', 'flax', 'livers', 'find', 'farfalle',
                'forest', 'calabrian', 'cottage', 'pasilla', 'thickness', 'griddle',
                'ladyfingers', 'pimiento', 'fuyu', 'branzino', 'drumettes', 'heatproof',
                'franks', 'trans', 'puréed', 'grains', 'together', 'sheeps',
                'dandelion', 'guinness', 'anaheim', 'farm', 'corned', 'stuffing',
                'proof', 'dust', 'resealable', 'buttering', 'nutella', 'individual',
                'liver', 'ramps', 'holland', 'kirsch', 'mesh', 'belly', 'lillet',
                'capacity', 'classic', 'lettuces', 'kind', 'tonic', 'steel', 'achiote',
                'minute', 'coco', 'vera', 'grey', 'gelato', 'pinot',
                'planks', 'machine', 'measured', 'twine', 'basket', 'globe', 'cajun',
                'xanthan', 'mâche', 'wash', 'lebanese', 'soybeans', 'cara', 'waffle',
                'sealable', 'deli', 'kieasa', 'mold', 'confit', 'picholine', 'bakers',
                'buttered', 'enough', 'large'
               ]

In [93]:
def text_cleaning2(text):
    text = str(text)
    text = text.replace(' lb ', '').replace(' Tbsp ', '').replace(' tsp ', '').replace(' oz ', '').replace(' cup ', '').replace(' to ', '')
    text_lst = text.split(', ')
    
#     words_lst = []
#     for ingredient in text_lst:
#         for w in ingredient.split():
#             words_lst.append(w)
    for ingredient in text_lst:        
        for sw in customised_sw:        
            ingredient = ingredient.replace(sw, '')
        
        
    
#     tokens_cleaned = [w for w in words_lst if not w in customised_sw]
    
    return text_lst

In [94]:
df['clean'] = df['ingredients'].apply(text_cleaning2)

In [95]:
df

,Unnamed: 0,recipe_title,stars,prep_times,cooking_times,review_count,difficulty_level,servings,description,specific_ingredients,ingredients,preference,clean
0,0,Chicken soup,4.4,15,35,180.0,Easy,Serves 4,Use up leftover chicken in this rustic soup with garlic yogurt. Best served with cheese scones,"1 tbsp ,olive oil, 2 ,onions,, chopped, 3 medium ,carrots,, chopped, 1 tbsp ,thyme leaves,, roughly chopped, 1.4l ,chicken stock, 300g ,leftover roast chicken,, shredded and skin removed, 200g ,frozen peas, 3 tbsp ,Greek yogurt, 1 ,garlic clove,, crushed, squeeze of ,lemon juice, cheese scone...","olive oil, onions, carrots, thyme leaves, leftover roast chicken, frozen peas, garlic clove,",british,"[olive oil, onions, carrots, thyme leaves, leftover roast chicken, frozen peas, garlic clove, ]"
1,1,Creamy pumpkin & lentil soup,4.0,15,35,74.0,Easy,Serves 4,"Whether you're carving a Halloween pumpkin or have picked up a squash from the supermarket, use the plentiful flesh and seeds in this soup","1 tbsp ,olive oil,, plus 1 tsp, 2 ,onions,, chopped, 2 ,garlic cloves,, chopped, approx 800g ,chopped pumpkin, flesh, plus the seeds, 100g ,split red lentil, ½ small pack ,thyme,, leaves picked, plus extra to serve, 1l ,hot vegetable stock, pinch of ,salt, and sugar, 50g ,crème fraîche,, plus ex...","olive oil, onions, garlic cloves, chopped pumpkin, thyme, crème fraîche,",british,"[olive oil, onions, garlic cloves, chopped pumpkin, thyme, crème fraîche, ]"
2,2,Citrus bars,4.0,20,35,18.0,More effort,Makes 18,Crisp shortbread biscuit base with a creamy citrus curd baked onto the top,"250g ,plain flour, 85g ,icing sugar,, plus extra for dusting, 175g ,butter,, cut into small pieces, 2 ,lemons, and 1 large orange, 4 ,large eggs, 400g ,caster sugar, 50g ,plain flour,","plain flour, icing sugar, butter, lemons, large eggs, caster sugar, plain flour,",british,"[plain flour, icing sugar, butter, lemons, large eggs, caster sugar, plain flour, ]"
3,3,Spiced plum & blackberry crumble,4.3,20,1 hr and 30,14.0,Easy,Serves 6,"Showcase fruit in a piping hot, oat-topped crumble delicately flavoured with cinnamon and star anise","140g ,plain flour, 140g ,butter,, cut into small dice, 85g ,soft brown sugar, 50g ,porridge oat, custard,, to serve, 1kg ,ripe plum, (about 14), halved and stoned, 1 tbsp ,soft brown sugar, 1 tbsp ,plain flour, 2 ,star anise, 1 tsp ,ground cinnamon, 1 tsp ,vanilla extract, juice ½ lemon, 300g ...","plain flour, butter, soft brown sugar, ripe plum, soft brown sugar, plain flour, star anise, ground cinnamon, vanilla extract, juice ½ lemon, blackberry,",british,"[plain flour, butter, soft brown sugar, ripe plum, soft brown sugar, plain flour, star anise, ground cinnamon, vanilla extract, juice ½ lemon, blackberry, ]"
4,4,Gooseberry & elderflower jam,n,5,30,7.0,More effort,Makes 3 x 340ml jars,"Preserve the taste of the season and use up a glut of gooseberries and elderflower cordial to make this simple, sweet spread","1kg ,gooseberries, 900g ,granulated white sugar, 100ml ,elderflower cordial,","gooseberries, granulated white sugar,",british,"[gooseberries, granulated white sugar, ]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9094,5398,"Dark chocolate, orange & ginger bara brith",3.6,10,1 hr and 15,3.0,Easy,Serves 8-10,"Try a twist on bara brith – the classic Welsh teacake – made with chocolate, orange and ginger. It's perfect for celebrating St David’s Day","3 strong black ,teabags, 300g ,dried mixed fruit, butter,, for the tin, 180g ,dark brown soft sugar, 4-5 balls of ,stem ginger,, roughly chopped, 100g ,dark chocolate,, chopped, 2 tsp ,marmalade, 2 drops of ,orange essence,, or use 1 orange, zested, 1 ,medium egg,, beaten, 230g ,self-raising fl...","butter, dark brown soft sugar, stem ginger, dark chocolate, medium egg, self-raising flour, ground cinnamon, ground ginger, dark chocolate, oranges,",welsh,"[butter, dark brown soft sugar, stem ginger, dark chocolate, medium egg, self-raising flour, ground cinnamon, ground g

In [ ]:
df['final_ing'] = df['Cleaned_Ingredients'].apply(text_cleaning)
df.head()

In [ ]:
def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

df['tokenized_ing'] = df['final_ing'].apply(tokenize_text)

In [ ]:
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))

In [ ]:
def clean_tokens(tokens):
    tokens_cleaned = [w for w in tokens if not w in stop_words]
    return tokens_cleaned

def removed_stopwords(tokens):
    removed_stopwords = [w for w in tokens if w in stop_words]
    return removed_stopwords

df['clean_tokens'] = df['tokenized_ing'].apply(clean_tokens)
df['removed_stopwords'] = df['tokenized_ing'].apply(removed_stopwords)

In [ ]:
new_ww = weighted_words.T

In [ ]:
len(clean_list)

In [ ]:
new_ww['score'] = new_ww.sum(axis=1)
new_ww[['score']].sort_values('score', ascending=False).head(50)
clean_list = new_ww.sort_values('score', ascending=False).index
clean_list[1499:1599]

In [ ]:
len(customised_sw)

In [ ]:
def removed_stopwords(tokens):
    removed_stopwords = [w for w in tokens if w not in customised_sw]
    return removed_stopwords

df['only_ing'] = df['clean_tokens'].apply(removed_stopwords)
df

In [ ]:
# words_to_keep = ['sugar', 'pepper', 'olive','butter', 'lemon', 'garlic', 'flour', 
#                  'cream','cloves', 'vinegar', 'water', 'milk', 'seeds','onion', 'vanilla', 
#                  'eggs', 'chicken', 'egg', 'wine', 'cheese', 'lime', 'parsley','orange', 
#                  'tomatoes', 'ginger', 'rice', 'chocolate', 'sprigs', 'cilantro','thyme', 
#                  'cinnamon', 'honey', 'mustard', 'fren', 'bread', 'clove', 'potatoes',
#                  'onions', 'coconut', 'mint', 'basil', 'beans', 'corn', 'parmesan', 'stems',
#                  'soda', 'sesame', 'celery', 'cumin', 'chiles', 'baby', 'apple', 'yogurt', 
#                  'scallions', 'rosemary', 'chives', 'chile','carrots', 'fennel', 'almonds', 
#                  'paprika', 'pork', 'cider', 'oregano','mushrooms', 'shallot', 'bacon', 
#                  'cayenne', 'shallots', 'tomato', 'beef','mayonnaise','nutmeg','dill', 
#                  'sherry', 'cornstarch', 'coriander','cherry', 'cocoa', 
#                  'peppers', 'peppercorns','peas', 'bean', 'sage', 'turkey', 
#                  'shrimp','wheat', 'spinach', 'almond','jalapeño', 'olives', 
#                  'strawberries', 'tarragon','walnuts', 'arugula', 'chili', 'rum']

In [ ]:
clean_tokens_texts = [' '.join(tokens) for tokens in df['clean_tokens']]
tf_idf_vectorizer = TfidfVectorizer(stop_words=customised_sw)

weighted_words = pd.DataFrame(tf_idf_vectorizer.fit_transform(clean_tokens_texts).toarray(),
                              columns=tf_idf_vectorizer.get_feature_names_out())

word_frequencies = weighted_words.sum().sort_values(ascending=False)

custom_word_frequencies = word_frequencies[word_frequencies.index.isin(customised_sw)]

print(custom_word_frequencies)

In [ ]:
weighted_words